In [6]:
import yfinance as yf

import pandas as pd

# Load your SP500.csv
csv_path = "/Users/raghav/Downloads/sp500tickers/SP500.csv"  # update if needed
df = pd.read_csv(csv_path)

# Get tickers as list
tickers = df["Symbol"].dropna().unique().tolist()

print("Total tickers:", len(tickers))
print("Sample:", tickers[:10])  # just to preview

Total tickers: 505
Sample: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'AYI', 'ADBE', 'AAP', 'AMD']


In [8]:
import yfinance as yf
import pandas as pd

# Download data for all tickers
data = yf.download(tickers, period="1d")["Close"]

# Get the last available row (latest prices)
latest_prices = data.iloc[-1]

# Convert to DataFrame
prices_df = latest_prices.reset_index()
prices_df.columns = ["Ticker", "Price"]

# ✅ Filter out tickers with missing prices (NaN)
prices_df = prices_df.dropna().reset_index(drop=True)

# ✅ Create new list of valid tickers
new_tickers = prices_df["Ticker"].tolist()

print(prices_df.head())
print(f"\nValid tickers: {len(new_tickers)} / {len(tickers)}")
print("\nSample new_tickers:", new_tickers[:10])

/var/folders/92/sfk96cdd5y5754wnlxc583zc0000gn/T/ipykernel_56909/939309258.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period="1d")["Close"]
[*********************100%***********************]  504 of 505 completed

85 Failed downloads:
['ANSS', 'HCN', 'FL', 'KORS', 'SNI', 'AET', 'CSRA', 'DPS', 'CBG', 'HES', 'GGP', 'COL', 'WYN', 'WBA', 'LUK', 'SCG', 'EVHC', 'ESRX', 'PCLN', 'TWX', 'NFX', 'BF.B', 'ANDV']: YFPricesMissingError('possibly delisted; no price data found  (period=1d)')
['ARNC', 'VAR', 'KSU', 'PDCO', 'GPS', 'DISCA', 'FLIR', 'UTX', 'CERN', 'SYMC', 'MON', 'XL', 'SRCL', 'COG', 'DISH', 'DWDP', 'NBL', 'DFS', 'BLL', 'CTXS', 'ETFC', 'CHK', 'DISCK', 'HCP', 'TIF', 'PKI', 'NLSN', 'CTL', 'CBS', 'FBHS', 'FISV', 'AGN', 'PXD', 'XLNX', 'ALXN', 'DRE', 'BRK.B', 'WLTW', 'RHT', 'BHGE', 'VIAB', 'JWN', 'RTN', 'ABC', 'TMK', 'PBCT', 'HRS', 'TSS', 'LLL', 'MYL', 'CELG', 'JEC', 'ADS', 'ANTM', 'ATVI', 'APC', 'MRO', 'CXO', 'JNPR', 'WRK',

  Ticker       Price
0      A  126.647499
1    AAL   12.415000
2    AAP   64.099998
3   AAPL  238.279999
4   ABBV  216.520004

Valid tickers: 420 / 505

Sample new_tickers: ['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM']


In [14]:
import pandas as pd
import numpy as np
import yfinance as yf

# ================== SETTINGS ==================
LOOKBACK_MONTHS = 12   # for SMA200 & 52w-high
AUTO_ADJUST = False    # be explicit to avoid surprises
# ==============================================

# ===== Helpers =====
def rsi(series, period: int = 14):
    delta = series.diff()
    up = np.where(delta > 0, delta, 0.0)
    down = np.where(delta < 0, -delta, 0.0)
    roll_up = pd.Series(up, index=series.index).ewm(alpha=1/period, adjust=False).mean()
    roll_down = pd.Series(down, index=series.index).ewm(alpha=1/period, adjust=False).mean()
    rs = roll_up / (roll_down.replace(0, np.nan))
    return 100 - (100 / (1 + rs))

def macd(series, fast=12, slow=26, signal=9):
    ema_fast = series.ewm(span=fast, adjust=False).mean()
    ema_slow = series.ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    hist = macd_line - signal_line
    return macd_line, signal_line, hist

def zscore(s, cap=3):
    z = (s - s.mean()) / (s.std(ddof=0) + 1e-9)
    return z.clip(-cap, cap) / cap  # scale to [-1,1]

# ===== Download OHLCV for all valid tickers (no batching) =====
data = yf.download(
    new_tickers,
    period=f"{LOOKBACK_MONTHS}mo",
    interval="1d",
    group_by="ticker",
    auto_adjust=AUTO_ADJUST,
    progress=False,
)

# --- Robust extraction of Close & High for any column shape ---
if isinstance(data.columns, pd.MultiIndex):
    lvl0 = data.columns.get_level_values(0)
    lvl1 = data.columns.get_level_values(1)
    if "Close" in lvl0 and "High" in lvl0:
        # Shape: (Field, Ticker)
        close = data["Close"]
        high  = data["High"]
    elif "Close" in lvl1 and "High" in lvl1:
        # Shape: (Ticker, Field)
        close = data.xs("Close", axis=1, level=1)
        high  = data.xs("High",  axis=1, level=1)
    else:
        raise RuntimeError("Could not find Close/High in downloaded data.")
else:
    # Single-ticker case → plain columns
    sym = new_tickers[0] if new_tickers else "TICKER"
    close = data[["Close"]].rename(columns={"Close": sym})
    high  = data[["High"]].rename(columns={"High": sym})

# Drop tickers with no data
close = close.dropna(axis=1, how="all")
high = high[close.columns]

if close.shape[1] == 0:
    raise RuntimeError("No usable tickers after download. Check new_tickers and connectivity.")

# ===== Build Features =====
prices = close.copy()
last = prices.tail(1).T.rename(columns={prices.index[-1]: "Price"})

sma50  = prices.rolling(50, min_periods=40).mean().tail(1).T.rename(columns={prices.index[-1]: "SMA50"})
sma200 = prices.rolling(200, min_periods=150).mean().tail(1).T.rename(columns={prices.index[-1]: "SMA200"})
rsi_14 = prices.apply(rsi, args=(14,)).tail(1).T.rename(columns={prices.index[-1]: "RSI14"})

macd_line, signal_line, hist = macd(prices)
macd_last   = macd_line.tail(1).T.rename(columns={prices.index[-1]: "MACD"})
signal_last = signal_line.tail(1).T.rename(columns={prices.index[-1]: "MACDSignal"})

ret_20 = prices.pct_change(20).tail(1).T.rename(columns={prices.index[-1]: "Ret20"})
ret_60 = prices.pct_change(60).tail(1).T.rename(columns={prices.index[-1]: "Ret60"})

# 52w-high proxy (use full window if <252 rows)
hi_lookback = prices.rolling(252, min_periods=min(60, prices.shape[0])).max()
hi_last = hi_lookback.tail(1).T.rename(columns={prices.index[-1]: "RollingHigh"})
dist_to_high = (last["Price"] / hi_last["RollingHigh"] - 1.0).to_frame(name="DistToHigh")

# Assemble feature table
feat = (
    last.join([sma50, sma200, rsi_14, macd_last, signal_last, ret_20, ret_60, dist_to_high])
        .replace([np.inf, -np.inf], np.nan)
        .dropna()  # ensure all features present
)

# ===== Scoring =====
W = {
    "ret20": 25,
    "ret60": 25,
    "dist_high": 15,
    "price_vs_sma50": 10,
    "sma50_vs_sma200": 10,
    "rsi": 15,
    "macd_signal": 10,
}

score = pd.Series(0.0, index=feat.index)
score += zscore(feat["Ret20"]) * W["ret20"]
score += zscore(feat["Ret60"]) * W["ret60"]
score += ((-feat["DistToHigh"]).clip(0, 1)) * W["dist_high"]
score += (feat["Price"] > feat["SMA50"]).astype(int) * W["price_vs_sma50"]
score += (feat["SMA50"] > feat["SMA200"]).astype(int) * W["sma50_vs_sma200"]
score += ((feat["RSI14"] - 50) / 50).clip(-1, 1) * W["rsi"]
score += (feat["MACD"] > feat["MACDSignal"]).astype(int) * W["macd_signal"]
score += (feat["MACD"] <= feat["MACDSignal"]).astype(int) * (-W["macd_signal"])

rank_df = feat.copy()
rank_df.insert(0, "Score", score.round(2))
rank_df = rank_df.sort_values("Score", ascending=False)

# ===== Output =====
top25 = rank_df.head(25)
bottom25 = rank_df.tail(25)

print("\n===== TOP 25 (Strongest) =====")
print(top25[["Score", "Price"]])

print("\n===== BOTTOM 25 (Weakest) =====")
print(bottom25[["Score", "Price"]])

rank_df.to_csv("sp500_ranked.csv")
print(f"\nSaved full ranking to sp500_ranked.csv with {len(rank_df)} tickers.")


===== TOP 25 (Strongest) =====
        Score       Price
Ticker                   
WDC     91.76  102.455002
STX     90.25  209.889999
GOOGL   89.77  251.509995
GOOG    89.38  251.779999
M       87.35   17.391600
MU      76.07  157.980103
NEM     70.13   79.470001
STI     68.68    4.690000
APA     68.22   24.715000
HP      67.91   21.760000
KSS     64.90   16.570101
VLO     60.69  162.009995
AAP     59.90   64.014999
EXPE    58.89  224.050003
NTAP    56.08  122.764999
GE      54.71  294.079010
TRIP    54.16   17.850000
TPR     52.29  108.044998
C       52.21  100.252403
KLAC    51.78  983.400024
ALLE    51.47  177.210007
BXP     51.22   78.410004
GS      50.86  785.830017
CMI     50.52  414.359985
MPC     50.11  183.955002

===== BOTTOM 25 (Weakest) =====
        Score       Price
Ticker                   
UDR    -21.58   37.480000
ZTS    -22.15  147.110001
PRU    -22.16  101.959999
CTAS   -22.20  199.024994
AJG    -22.61  289.709991
AMP    -23.61  485.970001
TXN    -24.46  177.625198

/var/folders/92/sfk96cdd5y5754wnlxc583zc0000gn/T/ipykernel_56909/2092873495.py:81: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret_20 = prices.pct_change(20).tail(1).T.rename(columns={prices.index[-1]: "Ret20"})
/var/folders/92/sfk96cdd5y5754wnlxc583zc0000gn/T/ipykernel_56909/2092873495.py:82: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret_60 = prices.pct_change(60).tail(1).T.rename(columns={prices.index[-1]: "Ret60"})


In [16]:
import alpaca_trade_api as tradeapi

# ================= SETTINGS =================
API_KEY = "AK0HVZ0FA9J3S85LNFF9"   
API_SECRET = "AV9HchlOsVcbbDjVCdtiYZoj8uS6nLrchUueesm9"  
BASE_URL = "https://api.alpaca.markets"

# Example tickers (replace with your own)
tickers = top25.index.tolist()
tickers1 = bottom25.index.tolist()

# =============================================

# Initialize API
api = tradeapi.REST(API_KEY, API_SECRET, BASE_URL, api_version='v2')

# BUY $1000 worth of each stock
for symbol in tickers:
    try:
        api.submit_order(
            symbol=symbol,
            notional=500,      # $1000 worth of stock
            side='buy',
            type='market',
            time_in_force='day'
        )
        print(f"✅ Order to BUY $1000 of {symbol} placed")
    except Exception as e:
        print(f"❌ Error placing order for {symbol}: {e}")

# SHORT approx $1000 worth of each stock
for symbol in tickers1:
    try:
        # Get latest price
        quote = api.get_latest_trade(symbol)
        price = quote.price

        # Calculate whole share qty to short
        qty = int(500 // price)   # floor division to ensure integer shares

        if qty > 0:
            api.submit_order(
                symbol=symbol,
                qty=qty,
                side='sell',       # short
                type='market',
                time_in_force='day'
            )
            print(f"✅ Order to SHORT {qty} shares of {symbol} (~$500)")
        else:
            print(f"⚠️ Skipped {symbol} (price too high for $500 short)")
    except Exception as e:
        print(f"❌ Error placing SHORT for {symbol}: {e}")

✅ Order to BUY $1000 of WDC placed
✅ Order to BUY $1000 of STX placed
✅ Order to BUY $1000 of GOOGL placed
✅ Order to BUY $1000 of GOOG placed
✅ Order to BUY $1000 of M placed
✅ Order to BUY $1000 of MU placed
✅ Order to BUY $1000 of NEM placed
❌ Error placing order for STI: asset "STI" is not fractionable
✅ Order to BUY $1000 of APA placed
✅ Order to BUY $1000 of HP placed
✅ Order to BUY $1000 of KSS placed
✅ Order to BUY $1000 of VLO placed
✅ Order to BUY $1000 of AAP placed
✅ Order to BUY $1000 of EXPE placed
✅ Order to BUY $1000 of NTAP placed
✅ Order to BUY $1000 of GE placed
✅ Order to BUY $1000 of TRIP placed
✅ Order to BUY $1000 of TPR placed
✅ Order to BUY $1000 of C placed
✅ Order to BUY $1000 of KLAC placed
✅ Order to BUY $1000 of ALLE placed
✅ Order to BUY $1000 of BXP placed
✅ Order to BUY $1000 of GS placed
✅ Order to BUY $1000 of CMI placed
✅ Order to BUY $1000 of MPC placed
❌ Error placing SHORT for UDR: account is not allowed to short
❌ Error placing SHORT for ZTS: acc

In [22]:
import json
print(json.dumps(account._raw, indent=2))


{
  "id": "ddd3024d-3a10-437b-8d85-1e0cf9049707",
  "admin_configurations": {
    "allow_instant_ach": true
  },
  "user_configurations": {
    "dtbp_check": "entry",
    "fractional_trading": true,
    "max_margin_multiplier": "4",
    "trade_confirm_email": "all"
  },
  "account_number": "651244584",
  "status": "ACTIVE",
  "crypto_status": "INACTIVE",
  "currency": "USD",
  "buying_power": "14154.09",
  "regt_buying_power": "14154.09",
  "daytrading_buying_power": "0",
  "effective_buying_power": "14154.09",
  "non_marginable_buying_power": "14154.09",
  "bod_dtbp": "0",
  "cash": "14154.09",
  "accrued_fees": "0",
  "portfolio_value": "26162.97",
  "pattern_day_trader": false,
  "trading_blocked": false,
  "transfers_blocked": false,
  "account_blocked": false,
  "created_at": "2024-10-29T14:21:35.888115Z",
  "trade_suspended_by_user": false,
  "multiplier": "1",
  "shorting_enabled": false,
  "equity": "26162.97",
  "last_equity": "225.36",
  "long_market_value": "12008.88",
  "sh